In [ ]:
!git clone https://github.com/rsoedarnadi/bdc-team4.git

fatal: destination path 'bdc-team4' already exists and is not an empty directory.


In [ ]:
#!gdown https://drive.google.com/u/0/uc?id=1AD_k3bK3qbKK9NSXLQb7pliuFcgpclWd&export=download

In [ ]:
#!unzip tr-ocr-fine-tuned2.zip

In [ ]:
!pip install transformers
!pip install accelerate -U
!pip install datasets
!pip install jiwer

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests
import os
import glob
import pandas as pd

In [ ]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-printed')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-printed')

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_df = pd.read_csv('./bdc-team4/retrain-labels.csv', delimiter=';')
test_df = pd.read_csv('./bdc-team4/DataTestLabelled.csv', delimiter=',')
train_df.head()

,Name of File,Labels
0,0_9.png,0
1,E_2.png,E
2,8_1.png,8
3,E_3.png,E
4,E_10.png,E


In [ ]:
test_df = test_df.drop(test_df.columns[0], axis=1)
train_df = train_df.rename(columns={train_df.columns[0]: "file_name", train_df.columns[1]: "text"}, inplace=False)
test_df = test_df.rename(columns={test_df.columns[1]: "text", test_df.columns[0]: "file_name"}, inplace=False)
train_df.head()

,file_name,text
0,0_9.png,0
1,E_2.png,E
2,8_1.png,8
3,E_3.png,E
4,E_10.png,E


In [ ]:
test_df.head()

,file_name,text
0,DataTest1.png,AD7034OE
1,DataTest2.png,A9388EX
2,DataTest3.png,B16TB
3,DataTest4.png,B1661TKZ
4,DataTest5.png,AD3772ABE


In [ ]:
import torch
from torch.utils.data import Dataset

class LPDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=9):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text,
                                          padding="max_length",
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [ ]:
train_dataset = LPDataset(root_dir=os.path.join(os.getcwd(),'bdc-team4/Retrain-Data/'),
                           df=train_df,
                           processor=processor)
eval_dataset = LPDataset(root_dir=os.path.join(os.getcwd(),'bdc-team4/Test-Data/'),
                           df=test_df,
                           processor=processor)

In [ ]:
encoding = train_dataset[3]
for k,v in encoding.items():
  print(k, v.shape)

pixel_values torch.Size([3, 384, 384])
labels torch.Size([9])


In [ ]:
image = Image.open(train_dataset.root_dir + train_df['file_name'][3]).convert("RGB")
image

In [ ]:
labels = encoding['labels']
labels[labels == -100] = processor.tokenizer.pad_token_id
label_str = processor.decode(labels, skip_special_tokens=True)
print(label_str)

E


In [ ]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 18
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    fp16=False,
    output_dir="./",
    logging_steps=2,
    save_steps=100,
    eval_steps=20,
)

In [ ]:
from datasets import load_metric

cer_metric = load_metric("cer")

<ipython-input-17-c81d87c6f9c2>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  cer_metric = load_metric("cer")


In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [ ]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/trocr/processing_trocr.py:134: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Cer
20,0.645900,0.600247,0.134211


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Step,Training Loss,Validation Loss,Cer
20,0.645900,0.600247,0.134211
40,1.003100,0.626403,0.109211
60,0.417600,0.765538,0.159211
80,0.315500,0.344646,0.085526
100,1.193100,0.730757,0.159211
120,0.369800,0.405785,0.088158
140,0.245300,0.370161,0.073684
160,0.394900,0.256542,0.072368
180,0.114500,0.242436,0.059211
200,0.220100,0.294414,0.055263


TrainOutput(global_step=336, training_loss=0.3833820835113459, metrics={'train_runtime': 1153.7768, 'train_samples_per_second': 2.314, 'train_steps_per_second': 0.291, 'total_flos': 1.9979214599592346e+18, 'train_loss': 0.3833820835113459, 'epoch': 3.0})

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

In [ ]:
model.save_pretrained("./tr-ocr-fine-tuned2")

In [ ]:
processor.save_pretrained("./tr-ocr-fine-tuned2")

In [ ]:
processor = TrOCRProcessor.from_pretrained('./tr-ocr-fine-tuned2', local_files_only=True)
model = VisionEncoderDecoderModel.from_pretrained('./tr-ocr-fine-tuned2', local_files_only=True)

In [ ]:
labels_csv = pd.read_csv('./bdc-team4/DataTestLabelled.csv', delimiter=',')
list_license_plates = labels_csv['Image Labels']
list_file_names = labels_csv['Name of File']

In [ ]:
generated_plate_numbers = []
for file in list_file_names:
    path = os.path.join(os.getcwd(), './bdc-team4/Test-Data', file)
    image = Image.open(path)
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    generated_plate_numbers.append(generated_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (18) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import re
for i, platenum in enumerate(generated_plate_numbers):
   generated_plate_numbers[i] = re.sub('[^A-Za-z0-9]+', '', platenum)

import numpy as np
def calculate_predicted_accuracy(names, actual_list, predicted_list):
    list_accuracy = []
    for name, actual_plate, predict_plate in zip(names, actual_list, predicted_list):
        accuracy = 0
        num_matches = 0
        if actual_plate == predict_plate:
            accuracy = 1.0
        else:
            for a, p in zip(actual_plate, predict_plate):
                if a == p:
                    num_matches += 1
            accuracy = round((num_matches / len(actual_plate)), 2)
        list_accuracy.append(accuracy)
    list_accuracy = np.asarray(list_accuracy)
    avg_value = np.average(list_accuracy)
    return avg_value

In [ ]:
calculate_predicted_accuracy(list_file_names, list_license_plates, generated_plate_numbers)

0.9675

In [ ]:
labels = pd.Series(generated_plate_numbers)
test_df = pd.concat([test_df, labels], axis=1)
test_df.to_csv('finetuned trocr testing results.csv')